### TP6 Gabriel García

1. Implementar un detector de Haar (vista frontal)
2. En esa ROI, detectar los ojos
3. Detectar si ambos ojos están abiertos o cerrados y mostrar el estado de la detección sobre el frame


In [1]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import cv2 as cv

Using matplotlib backend: Qt5Agg


In [3]:
# # Leemos los clasificadores pre-entrenados
#--------------------------------------------------------------------------------
face_cascade = cv.CascadeClassifier('haar_cascades/haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('haar_cascades/haarcascade_eye.xml')

In [4]:
cap = cv.VideoCapture(0)
counter = 0
ms_to_wait = 50
while(1):
    ret, frame = cap.read()
    if ret == True:

        # Pasamos la imagen a escala de grises y ecualizamos el frame
        #-------------------------------------
        grayImage = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        grayImage = cv.equalizeHist(grayImage)

        # Llamamos al clasificador de Haar (AdaBoost)
        #--------------------------------------------
        faces = face_cascade.detectMultiScale(grayImage, scaleFactor=1.1, minNeighbors=5, minSize=(100,130), maxSize=(200,400))
        
        # Recorro las caras encontradas y dibujo un rectángulo a su alrededor
        #------------------------------
        for (x,y,w,h) in faces:
            # Dibujo el rectángulo 
            cv.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2)

            # Definimos las ROIs en la imagen gris y color
            roi_gray = grayImage[y:y+h, x:x+w] 
            roi_color = frame[y:y+h, x:x+w]
            
            # Encuentro los ojos dentro de cada cara y los encierro en un rectángulo
            eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5, minSize=(int(w/6),int(h/6)), maxSize=(int(w/3),int(h/4)))
            
            # Recorro los ojos encontrados,  
            # si no encuentro espero 3 ciclos (150 ms) para confirmar si no es un parpadeo
            # Si pasan los 3 ciclos detecto ojos cerrados, y cuento los ms transcurridos. 
            
            try:
                total_eyes = eyes.shape[0]
                if eyes.shape[0] != 0:
                    for (ex,ey,ew,eh) in eyes:
                        counter = 0
                        cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),1)
                        # Escribimos sobre la imagen la cantidad de ojos abiertos
                        cv.rectangle(frame, (10, 2), (175,30), (255,255,255), -1)
                        cv.putText(frame, f"Ojos abiertos: {total_eyes}", (15, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6 , (0,0,0))
                
            except:
                counter += 1
                if counter > 3:
                    total_eyes = 0
                    # Escribimos sobre la imagen que los ojos están cerrados y el tiempo que permanecen así
                    cv.rectangle(frame, (10, 2), (320,30), (0,255,255), -1)
                    cv.putText(frame, f"Ojos cerrados durante {ms_to_wait*(counter-2)} ms", (15, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6 , (0,0,0))
            
        
        cv.imshow('Detector de ojos cerrados',frame)
        
        k = cv.waitKey(ms_to_wait) & 0xff
        if k == ord("q"):
            break
    else:
        break
cv.destroyAllWindows()
cap.release()